In [1]:
from urllib.request import urlopen
import json
import pandas as pd

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
    
# load a state level geojson file
with urlopen("https://raw.githubusercontent.com/PublicaMundi/MappingAPI/refs/heads/master/data/geojson/us-states.json") as response:
    states = json.load(response)

In [ ]:
# Load county FIPS codes
FIPS = pd.read_csv("../data/extras/US_FIPS_Codes.csv", dtype={"FIPS State": str, "FIPS County": str})
FIPS.head()

In [ ]:
datapath = "../data/suitability_scores/"
county_suitability = pd.read_csv(datapath + "suitability_scores_county.csv")
county_suitability.head()

In [4]:
# merge to get the FIPS codes
county_suitability = county_suitability.merge(FIPS, left_on=["State", "County Name"], right_on=["State", "County Name"])
county_suitability['fips'] = county_suitability['FIPS State'] + county_suitability['FIPS County']

In [5]:
# Solar Data
solar = pd.read_csv("../data/projects/solar/solar_raw.csv")

# Get weighted average of all factors

**Formulas:**
1. Avg of all factors = (sum of all factors) / (number of factors)
2. Gem Model weighted avg
    - 2 slope
    - 1 land cover
    - 2 population density
    - 2 habitat
    - 4 protected land
    - 1 distance to substation
    - 4 GHI (in the default GEM they used `Solar PV: 1-Axis Tracking Flat-Plate Collector`)

In [6]:
# Formula 1
factors = ['GHI', 'Protected_Land', 'Habitat', 'Slope', 'Population_Density', 'Distance_to_Substation', 'Land_Cover']

# sum the factors
formula_1 = county_suitability[factors].sum(axis=1).div(len(factors))
df_formula_1 = pd.DataFrame({"fips": county_suitability['fips'], "suitability": formula_1})

# Formula 2
factors_plus_weights = {
    'GHI': 4,
    'Protected_Land': 4,
    'Habitat': 2,
    'Slope': 2,
    'Population_Density': 2,
    'Distance_to_Substation': 1,
    'Land_Cover': 1
}

formula_2 = county_suitability.copy()
# Multiply each factor by its weight
for factor, weight in factors_plus_weights.items():
    formula_2[factor] = formula_2[factor] * weight
    
# sum the factors
formula_2 = formula_2[factors].sum(axis=1).div(sum(factors_plus_weights.values()))
df_formula_2 = pd.DataFrame({"fips": county_suitability['fips'], "suitability": formula_2})

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# custom color scale
pubu_color_scale = [
    (0.00, "#f7fcfd"),  # Very Light Blue (Unsuitable)
    (0.01, "#e0ecf4"), (0.40, "#e0ecf4"),  # Light Cyan
    (0.41, "#bfd3e6"), (0.50, "#bfd3e6"),  # Sky Blue
    (0.51, "#9ebcda"), (0.60, "#9ebcda"),  # Light Blue
    (0.61, "#8c96c6"), (0.70, "#8c96c6"),  # Moderate Blue
    (0.71, "#8c6bb1"), (0.80, "#8c6bb1"),  # Dark Teal
    (0.81, "#88419d"), (0.90, "#88419d"),  # Deep Purple-Blue
    (0.91, "#4a1486"), (1.00, "#4a1486")   # Very Dark Blue
]

# Create the choropleth map
fig = px.choropleth(
    df_formula_2,
    geojson=counties,
    locations='fips',  # Column in df that matches GeoJSON IDs
    color='suitability',  # Column to plot
    color_continuous_scale=pubu_color_scale,  # Custom color scale
    range_color=(0, 100),
    scope="usa",  # Focus on the United States    
)

fig.update_traces(marker_line_width=0)  # Remove county borders

# Overlay state boundaries
fig.add_trace(go.Choropleth(
    geojson=states,  # Use the state-level GeoJSON
    locations=[feature["id"] for feature in states["features"]],  # State IDs
    z=[0] * len(states["features"]),  # Dummy data to avoid coloring
    colorscale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,0,0,0)"]],  # Transparent fill
    showscale=False,
    marker=dict(line=dict(color='black', width=1.5)),  # Black state boundaries
))

# Update layout and map appearance
fig.update_geos(
    fitbounds="locations",  # Auto-zoom based on data
    visible=False  # Hide base map
)

fig.update_layout(
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)

# add state points px.scatter_geo(solar, lat="latitude", lon="longitude", scope="usa", color="statename")
len_scatter = len(px.scatter_geo(solar, lat="latitude", lon="longitude", scope="usa", size="total_mw").data)
for i in range(len_scatter):
    fig.add_trace(
        px.scatter_geo(solar, lat="latitude", lon="longitude", scope="usa", size="total_mw", color_discrete_sequence=px.colors.qualitative.Light24).data[i]
    )

# Don't show the legend for the scatter points
fig.update_layout(showlegend=False)


fig.show()